### Welcome to Week 6 Day 3!

Let's experiment with a bunch more MCP Servers

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

### The first type of MCP Server: runs locally, everything local

Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


---


So, the first type of MCP server that I mentioned is one that you run locally, and everything it does is local. Again, remembering, we downloaded it from a public repo, somewhere public, but we're going to run it. Server will be applied locally. And we're going to pick another JavaScript one, a node one, and I want to pick this one, which is great. It is a way to give our agent memory, and to give it knowledge graph-based memory, a special kind of memory that understands about entities and observations about those entities and relationships between those entities. Memory is a hot topic, and people often ask me questions about memory, and they think of memory like it's kind of one construct, perhaps because LangChain thinks of it that way, but like there is one memory. But in this MCP age, we think of memory just as being another set of tools that we can equip our LLM with. And in fact, it could have many different types of tools relating to memory. This in particular is giving it the ability to store information about things, observations, and relationships. We could give it this, and we could give it other kinds of context. All of this ends up just being information that it can request by tools or that becomes available in its prompts. And so they're just different techniques to give the LLM more context. Anyway, so these are the parameters. Remember, you will start by specifying the parameters of your MCP server. And in this case, it's MPX because we're running a node version, and we're going to be running this version called libsql, which is a version, a memory that stores to a SQL-like database. I've actually given a link here to one that this is closely based on, which is one that gives you access to a JSON file, but doesn't actually store it to SQL. And I found that this is a little bit less stable. This is a better version of it that I prefer. It also allows you to specify a directory and a name for your memory. And this allows you to have different memory stores for different agents. So I find that quite handy. So I'm saying that look in somewhere called memory and look for a database called ed. And I believe it might already exist. It does. So why don't we just delete this so that it doesn't have any memory to start with? And move that to trash, starting from nothing. So if you have ed in a folder there, then delete it. If you don't have a folder, then create one called memory.


In [2]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/ed.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

All right. So first of all, let's just find out what tools do we have associated with this. We have tools create entities, search nodes, read graph, create relations, delete entity, and delete relation. So we're allowing it to build this kind of connectivity between the things it wants to remember.


So now we have instructions. You use your entity tools as persistent memory to store and record information about your conversations. My name is ed. I'm an LLM engineer. I'm teaching a course about agents, including the incredible MCP protocol. And I say something about MCP. 

In [3]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Ed. I'm an LLM engineer. I'm teaching a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4.1-mini"

And I give it a model. And let's see how this gets on. So as before, this is hopefully a pattern that you're quite familiar with. We use this context manager. We have MCP server studio, which we know is going to create the MCP client. We pass in the parameters. We give this timeout of 30 seconds. And then, as usual, we create our agent with instructions, with the model, and now with the MCP server. And now we call runner.run with the agent and our request. And let's see what happens. Armed with these tools, is it able to take this piece of information about me and do something with it? 

In [4]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Nice to meet you, Ed! It sounds like your course on AI Agents, especially with a focus on the MCP protocol, is covering some exciting and advanced topics. If you need help preparing content, explanations, or anything related to your teaching, just let me know!

So it says, nice to meet you, Ed. It's great to know you're teaching a course about AI agents and covering MCP protocol, blah, blah, blah, blah, blah. And if you look over on the left, it has created a file, `memory/ed.db` 

---

So now for the follow-up question. We're coming back. We're creating another agent. We're giving it instructions. We're asking for the same model, and we're giving it the same MCP server with access to the memory. And we're saying, my name's Ed. What do you know about me? And we'll see what this MCP server is able to do.

In [5]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Ed. What do you know about me?")
    display(Markdown(result.final_output))

I know that you are Ed, and you are an LLM engineer. You are also teaching a course about AI Agents. Is there anything specific you would like to add or update about yourself?

 I know that you're Ed, an LLM engineer. We teach you a course about AI agents, and in this course, we teach you about the MCP protocol. So there you go. This is a quick, simple example, but it shows you how we were so easily able to equip our agent with a memory that's able to handle relationships between things. And so this is just a great starting example, and we'll be using this memory in the future as well, and it could be handy for any of your projects too. And of course, it's always a good idea to check the trace, which we should do. Come on in. 
 
 ![](../img/88.png)
 
 And conversation with our agent is right here, and you will see that it called the search nodes. It looked into the query for Ed, and it got back here JSON structure that reflects the entity type, the observations, and so on. And so you can look through this and get a sense for how the memory works, and you can use this to try and build up some more sophisticated memory about the different parts of your conversation that you want your agent to remember. Okay. Now we're going to go on to another MCP tool, and this time it's going to be the second type, which is the type that runs locally but uses the Internet, which we've already done with Vetch and with Playbytes. But let's do another, and a handy one.

### Check the trace:

https://platform.openai.com/traces

### The 2nd type of MCP server - runs locally, calls a web service

### Brave Search - apologies - this will need another API key! But it's free again.

https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

In [6]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

mcp_tools

ValidationError: 1 validation error for StdioServerParameters
env.BRAVE_API_KEY
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [7]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### As usual, check out the trace:

https://platform.openai.com/traces

## And now the third type: running remotely

It's actually really hard to find a "remote MCP server" aka "hosted MCP server" aka "managed MCP server".

It's not a common model for using or sharing MCP servers, and there isn't a standard way to discover remote MCP servers.

Anthropic lists some remote MCP servers, but these are for paid applications with business users:

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlare has tooling for you to create and deploy your own remote MCP servers, but this does not seem to be a common practice:

https://developers.cloudflare.com/agents/guides/remote-mcp-server/


# And back to the 2nd type: the Polygon.io MCP Server

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">PLEASE READ!!-</h2>
            <span style="color:#ff7800;">This service for financial market data has both a FREE plan and a PAID plan, and we can use either depending on your appetite.
            </span>
        </td>
    </tr>
</table>

## NEW SECTION: Introducing polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

### Polygon.io Part 1: Polygon.io free service (the paid will be totally optional, of course!)

1. Please sign up for polygon.io (top right)  
2. Once signed in, please select "Keys" in the left hand navigation
3. Press the blue "New Key" button
4. Copy the key name
5. Edit your .env file and add the row:

`POLYGON_API_KEY=xxxx`

In [3]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [ ]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

### Wrapped into a python module that caches end of day prices

I've made a python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.


In [ ]:
from market import get_share_price
get_share_price("AAPL")

In [ ]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

### And I've made this into an MCP Server

Just as we did with accounts.py; see `market_server.py`

In [ ]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools

### Let's try it out!

Hopefully gpt-4o-mini is smart enough to know that the symbol for Apple is AAPL

In [ ]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Polygon.io Part 2: Paid Plan - Totally Optional!

If you are interested, you can subscribe to the monthly plan to get more up to date market data, and unlimited API calls.

If you do wish to do this, then it also makes sense to use the full MCP server that Polygon.io has released, to take advantage of all their functionality.



In [ ]:

params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools


### Wow that's a lot of tools!

Let's try them out - hopefully the sheer number of tools doesn't overwhelm gpt-4o-mini!

With the $29 monthly plan, we don't have access to some of the APIs, so I've needed to specify which APIs can be called.

If you've splashed out on a bigger plan, feel free to remove my extra constraint..

In [ ]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Setting up your .env file

If you do decide to have a paid plan, please add this to your .env file to indicate:

`POLYGON_PLAN=paid`

And if you decide to go all the way for the realtime API, then please do:

`POLYGON_PLAN=realtime`

In [ ]:
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("You've chosen to subscribe to the paid Polygon plan, so the code will look at prices on a 15 min delay")
elif is_realtime_polygon:
    print("Wowzer - you've chosen to subscribe to the realtime Polygon plan, so the code will look at realtime prices")
else:
    print("According to your .env file, you've chosen to subscribe to the free Polygon plan, so the code will look at EOD prices")

## And that's it for today!

I've removed the part of this lab that uses the "Financial Datasets" mcp server, because it's inferior - more expensive with fewer APIs.

And this way we get to use the same provider for Free and Paid APIs.

But if you want to see the code, just look in the git history for a prior version.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Explore MCP server marketplaces and integrate your own, using all 3 approaches.
            </span>
        </td>
    </tr>
</table>